# Fine-tuning de GPT-2 com PEFT (LoRA) no Dataset AmazonTitles-1.3MM

Este notebook demonstra o processo de fine-tuning do modelo GPT-2 usando o método PEFT (Parameter-Efficient Fine-Tuning) com LoRA (Low-Rank Adaptation) no dataset AmazonTitles-1.3MM. Utilizaremos as bibliotecas Hugging Face Transformers, Datasets e PEFT para este processo.

## Instalação das Dependências

Primeiro, vamos instalar as bibliotecas necessárias.

In [ ]:
!pip install transformers datasets torch peft

## Importação das Bibliotecas

In [ ]:
import torch
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import DataCollatorForLanguageModeling
from peft import get_peft_model, LoraConfig, TaskType, PeftModel

## Carregamento e Preparação do Dataset

Nesta seção, carregaremos o dataset AmazonTitles-1.3MM e o prepararemos para o treinamento.

In [ ]:
# Carregando o dataset
# Nota: Você precisará fazer upload do arquivo trn.json para o Colab ou usar um link para o arquivo
dataset = load_dataset('json', data_files={'train': 'path/to/your/dataset/trn.json'})

# Função para preparar os dados
def prepare_data(examples):
    return {
        'text': [f"Title: {title}\nContent: {content}\nResponse:" for title, content in zip(examples['title'], examples['content'])]
    }

# Aplicando a preparação dos dados
dataset = dataset.map(prepare_data, batched=True, remove_columns=dataset['train'].column_names)

# Dividindo o dataset em treino e validação
dataset = dataset['train'].train_test_split(test_size=0.1)

print(f"Tamanho do conjunto de treinamento: {len(dataset['train'])}")
print(f"Tamanho do conjunto de validação: {len(dataset['test'])}")

## Configuração do Modelo e Tokenizador

In [ ]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Configurando o tokenizador
tokenizer.pad_token = tokenizer.eos_token

# Função para tokenizar os dados
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

# Aplicando a tokenização
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])

# Preparando o data collator
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

## Configuração do PEFT com LoRA

In [ ]:
# Configuração do LoRA
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)

# Aplicar PEFT ao modelo
model = get_peft_model(model, peft_config)

## Configuração do Treinamento

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
)

## Treinamento do Modelo

In [ ]:
trainer.train()

## Salvando o Modelo

In [ ]:
peft_model_path = "./peft_model"
trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

# Para baixar o modelo treinado do Colab
from google.colab import files
!zip -r peft_model.zip peft_model
files.download('peft_model.zip')

## Carregando o Modelo para Inferência

In [ ]:
def load_peft_model(model_path):
    config = PeftModel.from_pretrained(model_path, 'gpt2')
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    model = PeftModel.from_pretrained(model, model_path)
    return model

model = load_peft_model(peft_model_path)
model.eval()

## Função para Gerar Respostas

In [ ]:
def generate_response(title, content):
    input_text = f"Title: {title}\nContent: {content}\nResponse:"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    
    with torch.no_grad():
        output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2, top_k=50, top_p=0.95)
    
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    response = generated_text.split('Response:')[-1].strip()
    
    return response

# Exemplo de uso
title = "Smartphone de última geração"
content = "Este smartphone possui câmera de alta resolução, bateria de longa duração e processador potente."
response = generate_response(title, content)
print(f"Título: {title}")
print(f"Conteúdo: {content}")
print(f"Resposta gerada: {response}")

## Conclusão

Este notebook demonstrou o processo de fine-tuning do modelo GPT-2 usando PEFT com LoRA no dataset AmazonTitles-1.3MM. O modelo treinado pode ser usado para gerar respostas baseadas em títulos e descrições de produtos.

Pontos importantes:
1. PEFT com LoRA permite um fine-tuning eficiente em termos de memória e computação.
2. O modelo treinado mantém o conhecimento geral do GPT-2 enquanto se adapta à tarefa específica.
3. O processo de treinamento é mais rápido e requer menos recursos comparado ao fine-tuning tradicional.
4. O modelo resultante é menor em tamanho, facilitando o armazenamento e a distribuição.

Para melhorar ainda mais:
- Experimente com diferentes configurações de LoRA (r, alpha, dropout).
- Ajuste os hiperparâmetros de treinamento para otimizar o desempenho.
- Considere usar técnicas de avaliação mais robustas para medir a qualidade das respostas geradas.